In [1]:
from multiprocessing import Process
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

import cv2
import netCDF4 as nc
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import json

from subprocess import run, DEVNULL

from torch.nn import Sequential, Sigmoid
from torch import from_numpy, inference_mode
from torch.nn import DataParallel

from monai.networks.nets import SwinUNETR
from monai.inferers import sliding_window_inference

from pytorch_lightning import LightningModule

from zipfile import ZipFile
from zipfile import BadZipFile
from zipfile import is_zipfile

import xmltodict

import time

import os
from os.path import join, exists, getsize, isfile

In [2]:
class DownloadSARSentinel:
    def __init__(self, email, 
             password,
             product_list,
             satellite="sentinel_1"
                ):
        self.password = password
        self.email = email
        self.product_list = product_list
        self.satellite = satellite
        self.raw_folder = None
        self._query_result = None

    def query_result(self):
        if not isinstance(self._query_result, pd.DataFrame):
            self._query_result = self.query_sar()
        return self._query_result
        
    def query_sar(self):
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'S1A') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-21T00:00:00.000Z"
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{name}')"
        query_result = pd.DataFrame()

        for product in self.product_list:
            json = requests.get(API_URL_NAME.format(name=product)).json()
            response_result = pd.DataFrame.from_dict(json['value'])
            
            if not response_result.empty:
                print("Found:", product)
                query_result = pd.concat([query_result, response_result])   
                time.sleep(0.5)
            else:
                print("Could not find: ", product)

        return query_result
        
    @staticmethod
    def create_folder(folder_path, original_name):
        if not folder_path:
            folder_path = join(os.getcwd(), original_name)
            if not exists(folder_path):
                os.mkdir(folder_path)

        assert exists(folder_path), f"Path not found: {folder_path}"
        return folder_path

    @staticmethod
    def is_downloaded(sar_name, folder):
        sar_path = join(folder, sar_name + ".zip")
        if exists(sar_path):
            #if getsize(sar_path) > 5E9: #ver o tamanho do arquivo para baixar arquivos incompletos
            return True
        return False

    def return_headers(self):
        token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'password',
            'username': self.email,
            'password': self.password,
            'client_id': 'cdse-public'
        }
        token_response = requests.post(token_url, headers=headers, data=data).json()
        token_url = token_response["access_token"]
        download_headers = {"Authorization": f"Bearer {token_url}"}
        return download_headers

    def download_products(self, folder, overwrite=False):
        download_url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

        for index, product in self.query_result().iterrows():
            path_product = join(folder, product["Name"] + ".zip")               
            if not DownloadSARSentinel.is_downloaded(product["Name"], folder) or overwrite:
                session = requests.Session()
                session.headers.update(self.return_headers())
                response = session.get(download_url.format(product_id=product["Id"]), headers=self.return_headers(), stream=True)
                total_size = int(response.headers.get('Content-Length', 0))
                progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f'Downloading: {product["Name"]}', leave=True)
                progress_bar.reset()

                with open(path_product, "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                            progress_bar.update(len(chunk))
                progress_bar.reset()




        

In [9]:
QUERY_CSV_PATH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/New_sar_img.csv"
FILE_NAME_COLUMN = "NOME DO ARQUIVO"
RAW_SAFE_FILE = "/mnt/camobi_process/new_data/new_SAR_img"
UNZIP_SAFE_FILE = "/mnt/camobi_process/new_data/unzip_sar_img"
query_sar = pd.read_csv(QUERY_CSV_PATH, header=0)[FILE_NAME_COLUMN]

sar_list = []

for sar_name in query_sar:
    sar_path = join(UNZIP_SAFE_FILE, sar_name + ".SAFE")
    if not exists(sar_path):
        sar_list.append(sar_name)
#print(len(sar_list))

new_products = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=sar_list)
#new_products.query_result()

new_products.download_products(folder="/mnt/camobi_process/new_data/new_SAR_img")

KeyboardInterrupt: 

In [181]:
downlaod_data = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=list(query_sar))

In [3]:
class SentinelProduct:
    def __init__(self, name, file_path, nc_graph_path,  rect_corners=None):
        self.name = name
        self.simple_name = name.split(".")[0][-7:-4]
        self.rect_corners = rect_corners
        self.file_path = file_path
        self.nc_graph_path = nc_graph_path
        self.unzip_path = None
        self.netcdf_path = None

    def unzip(self, unzip_folder):
        assert exists(unzip_folder), f"Folder \"{unzip_folder}\" does not exist!"
        self.unzip_path = join(unzip_folder, self.name + ".SAFE") 
        try:
            print("Unziping:", self.name)
            with ZipFile(self.file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_folder)
        except BadZipFile:
            print(f"Imposible to unzip: {self.name}. file is incomplete or corrupted!")

    def edit_zip_to_nc(self, netcdf_folder):
        with open(self.nc_graph_path) as arquivo:
            dados = xmltodict.parse(arquivo.read())

        if self.unzip_path:
            input_path = self.unzip_path
        else:
            input_path = self.file_path
        
        self.netcdf_path = join(netcdf_folder, self.simple_name + ".nc")
        print(self.netcdf_path)
        

        dados['graph']['node'][0]['parameters']['file'] = input_path 
        dados['graph']['node'][-1]['parameters']['file'] = self.netcdf_path 

        print(dados['graph']['node'][0]['parameters']['file'])


        with open(self.nc_graph_path, 'w') as arquivo:
            arquivo.write(xmltodict.unparse(dados, pretty=True))

    def convert_to_netcdf4(self, gpt_path, netcdf_folder):
        self.edit_zip_to_nc(netcdf_folder)
        shell = run([gpt_path, self.nc_graph_path])#, stdout=DEVNULL, stderr=DEVNULL)



In [8]:
SAR_TO_NC_GRAPH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/graphs/ZIP_to_NC.xml"
PATH_TO_GPT = "/home/camobi/snap/bin/gpt"
PRODUCT_FOLDER = "/mnt/camobi_process/new_data/new_SAR_img"
UNZIP_FOLDER = "/mnt/camobi_process/new_data/unzip_sar_img"
NETCDF_FOLDER = "/mnt/camobi_process/new_data/images_nc"

product_list = []
count = 0

for product in os.listdir(UNZIP_FOLDER):
    abv_name = product[-9:-5] + ".nc"
    test = join(NETCDF_FOLDER, abv_name)
    if "SAFE" in product and not exists(join(NETCDF_FOLDER, abv_name)):
        count += 1
        product_name = product[:-5]
        print(product_name)
        product_path = join(UNZIP_FOLDER, product + "/")
        product_file = SentinelProduct(product_name, product_path, SAR_TO_NC_GRAPH)
        product_list.append(product_file)
count
#for product in tqdm(product_list):
    #product.unzip(UNZIP_FOLDER)

S1A_IW_SLC__1SDV_20170709T000134_20170709T000201_017387_01D0AA_205D
S1B_IW_SLC__1SDV_20211121T142946_20211121T143013_029687_038B19_5510
S1A_IW_SLC__1SDV_20210805T000201_20210805T000228_039087_049CB9_D205
S1A_IW_SLC__1SDV_20210612T235346_20210612T235415_038314_048582_23F3
S1B_IW_SLC__1SDV_20211003T014926_20211003T014953_028965_0374DA_4688
S1B_IW_SLC__1SDV_20190325T081332_20190325T081359_015508_01D0FD_EBAF


6

In [185]:
for product in product_list:
    product.convert_to_netcdf4(PATH_TO_GPT, NETCDF_FOLDER)

product_list

/mnt/camobi_process/new_data/images_nc/S1A_IW_SLC__1SDV_20170709T000134_20170709T000201_017387_01D0AA_205D.nc
/mnt/camobi_process/new_data/unzip_sar_img/S1A_IW_SLC__1SDV_20170709T000134_20170709T000201_017387_01D0AA_205D.SAFE/


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.2.1 from distribution will be used. (f0)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.2.1 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.2.1 set to be used by SNAP.


Executing processing graph


INFO: org.hsqldb.persist.Logger: dataFileCache open start


OpenSearch: https://scihub.copernicus.eu/gnss/search?q=platformname:Sentinel-1 AND platformnumber:A AND producttype:AUX_POEORB AND beginposition:[2017-07-01T00:00:000Z TO 2017-07-31T24:00:000Z]


...

OpenSearch: https://scihub.copernicus.eu/gnss/search?q=platformname:Sentinel-1 AND platformnumber:B AND producttype:AUX_RESORB AND beginposition:[2021-11-01T00:00:000Z TO 2021-11-31T24:00:000Z]


Orbit files may be downloaded from https://scihub.copernicus.eu/gnss/odata/v1/
and placed in /home/los/.snap/auxdata/Orbits/Sentinel-1/POEORB/S1B/2021/11
6482 [main] INFO serverStartup - Nc4Iosp: NetCDF-4 C library loaded (jna_path='/home/los/.snap/auxdata/netcdf_natives/9.0.0/amd64', libname='netcdf').
6486 [main] INFO serverStartup - NetcdfLoader: set log level: old=0 new=0
6486 [main] INFO serverStartup - Nc4Iosp: set log level: old=0 new=0
org.esa.snap.core.gpf.OperatorException: Not able to write product file: '/mnt/camobi_process/new_data/images_nc/S1B_IW_SLC__1SDV_20211121T142946_20211121T143013_029687_038B19_5510.nc'
	at org.esa.snap.core.gpf.common.WriteOp.doExecute(WriteOp.java:319)
	at org.esa.snap.core.gpf.internal.OperatorContext.executeOperator(OperatorContext.java:1300)
	at org.esa.snap.core.gpf.Operator.execute(Operator.java:153)
	at org.esa.snap.core.gpf.graph.GraphProcessor.executeGraph(GraphProcessor.java:198)
	at org.esa.snap.core.gpf.graph.GraphProcessor.executeGra

90% done.
#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x00007f66b75214db, pid=2224306, tid=0x00007f681c227700
#
# JRE version: OpenJDK Runtime Environment (Zulu 8.44.0.13-CA-linux64) (8.0_242-b20) (build 1.8.0_242-b20)
# Java VM: OpenJDK 64-Bit Server VM (25.242-b20 mixed mode linux-amd64 )
# Problematic frame:
# C  [libhdf5.so.9.0.0+0xcf4db]  H5F_close+0x22
#
# Failed to write core dump. Core dumps have been disabled. To enable core dumping, try "ulimit -c unlimited" before starting Java again
#
# An error report file with more information is saved as:
# /mnt/camobi_2/PHMG/Sentinel_Acquisition/hs_err_pid2224306.log
#
# If you would like to submit a bug report, please visit:
#   http://www.azulsystems.com/support/
#
/mnt/camobi_process/new_data/images_nc/S1A_IW_SLC__1SDV_20210805T000201_20210805T000228_039087_049CB9_D205.nc
/mnt/camobi_process/new_data/unzip_sar_img/S1A_IW_SLC__1SDV_20210805T000201_20210805T000228_039087_049CB9_D205.SAF

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.2.1 from distribution will be used. (f0)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.2.1 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


KeyboardInterrupt: 

In [4]:
class PedroNet(LightningModule): #out_channels = numero de classes
    def __init__(self, img_size, lr,
                 depths=(2, 2, 2, 2), 
                 num_heads=(3, 6, 12, 24), 
                 feature_size=24, 
                 norm_name='instance', 
                 drop_rate=0.0, 
                 attn_drop_rate=0.0, 
                 dropout_path_rate=0.0, 
                 normalize=True, 
                 use_checkpoint=False, 
                 downsample='merging', 
                 use_v2=False 
                 ):
        super().__init__()
        self.model = Sequential(SwinUNETR(spatial_dims=2,
                                    in_channels=1,
                                    out_channels=1,
                                    depths=depths,
                                    img_size=img_size,
                                    feature_size=feature_size,
                                    drop_rate=drop_rate,
                                    num_heads=num_heads,
                                    norm_name=norm_name,
                                    attn_drop_rate=attn_drop_rate,
                                    dropout_path_rate=dropout_path_rate,
                                    normalize=normalize,
                                    use_checkpoint=use_checkpoint,
                                    downsample=downsample,
                                    #use_v2=use_v2, apenas para versões mais recentes
                                    ))
    def forward(self, x):
        return self.model(x)
    
    def netcdf_inferece(self, nc_product):
        torch_img = from_numpy(np.asarray(nc_product.nc_img)).to("cuda")
        torch_img = torch_img.unsqueeze(0).unsqueeze(0)

        with inference_mode():
            model_img = sliding_window_inference(torch_img, 
                                            roi_size=(512),
                                            sw_batch_size=20, 
                                            predictor=DataParallel(self.model), 
                                            mode='constant',
                                            overlap=0.8,
                                            progress=True
                                            )

            sigmoid_fn = Sigmoid()
            model_img = sigmoid_fn(model_img)
            model_img = model_img.to("cpu").squeeze()
        return model_img

class NetcdfProduct:
    def __init__(self, product, image_variable='Sigma0_VV_db'):
        self.product = product
        self.name = product.filepath().split("/")[-1].split(".")[0]
        self.image_variable = image_variable
        self.nc_img = product.variables[self.image_variable][:]
    
    def __str__(self):
        return self.name

    def create_img(self, folder_path):
        img_path = join(folder_path, self.name + ".png")
        normalized_img = ((self.nc_img - np.min(self.nc_img)) / (np.max(self.nc_img) - np.min(self.nc_img))) * 255
        cv2.imwrite(img_path, normalized_img)
    
    @staticmethod
    def create_polygons(mask):
        edited_contours = []
        binary_image = np.array(mask)
        
        contours, hierarchy = cv2.findContours(binary_image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for poly in contours:
            if len(poly) > 50:
                approx = cv2.approxPolyDP(poly, 0.8, True)
                approx = np.squeeze(approx)
                edited_contours.append(approx)
        return edited_contours
    
    @staticmethod
    def mask_to_bool(probs, threshold=0.5):
        mask = np.asarray(probs)
        mask[mask > threshold] = True
        mask[mask < threshold] = False
        return mask

    def probs_to_labelme(self, probs, folder, threshold=0.5):
        mask = NetcdfProduct.mask_to_bool(probs, threshold)
        masked_polygons = NetcdfProduct.create_polygons(mask)
        labelme_format = {"version": "5.1.1",
                          "flags": {},
                          "shapes": [],
                          "imagePath": f"..\\Sar_img\\{self.name}.png",  # Update with your image filename
                          "imageData": None,
                          "imageHeight": self.product.dimensions["y"].size,
                          "imageWidth": self.product.dimensions["x"].size
                          }

        for patch in masked_polygons:
            labelme_format["shapes"].append({
                "label": "oil", 
                "points": patch.squeeze().tolist(),
                "group_id": None,
                "description": "",
                "shape_type": "polygon",
                "flags": {}
            })

        auto_labels_path = join(folder, f"{self.name}.json")
        with open(auto_labels_path, 'w') as json_file:
            json.dump(labelme_format, json_file, indent=2)
        print(f"Label \"{self.name}.json\" created!")


In [12]:
NETCDF_PRODUCT_PATH = "/mnt/camobi_process/new_data/images_nc"

list_nc_product = []

for nc_data in tqdm(os.listdir(NETCDF_PRODUCT_PATH)):
    nc_data_path = join(NETCDF_PRODUCT_PATH, nc_data)
    nc_img = nc.Dataset(nc_data_path, 'r')
    netcdf_sar = NetcdfProduct(nc_img)
    list_nc_product.append(netcdf_sar)
    #netcdf_sar.create_img("/mnt/camobi_2/PHMG/Sentinel_Acquisition/img_folder")

100%|██████████| 69/69 [03:19<00:00,  2.88s/it]


In [16]:
LABELS_FOLDER = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/auto_labels"
WEIGHTS_MODEL = "/mnt/camobi_2/PHMG/PedroSwinNet/Model_512Img_24Feature_(2, 2, 2, 2)depths_0.0attnDrop_(3, 6, 12, 24)Heads_30.000000Lr_0drop_v1/model-Val_loss=0.003714-Val_Precision=0.973-Recall=0.971-Val_F1_Score=0.972.ckpt"
loaded_model = PedroNet.load_from_checkpoint(WEIGHTS_MODEL)


for nc_image in tqdm(list_nc_product):
    probs = loaded_model.netcdf_inferece(nc_image)
    nc_image.probs_to_labelme(probs, LABELS_FOLDER)

  1%|▏         | 1/69 [01:00<1:09:00, 60.89s/it]

Label "A8A3.json" created!


  3%|▎         | 2/69 [01:10<34:09, 30.60s/it]  

Label "2EBC.json" created!


  4%|▍         | 3/69 [01:12<19:22, 17.62s/it]

...

Label "F8D3.json" created!


 10%|█         | 7/69 [05:36<59:37, 57.71s/it]

Label "C094.json" created!


 10%|█         | 7/69 [06:35<58:26, 56.56s/it]


KeyboardInterrupt: 

In [303]:

#plt.imshow(test_mask)
test_mask_1 = np.array(test_mask)
test_mask_1[netcdf_sar.nc_img.mask] = np.nan
#plt.imshow(test_mask_1)
netcdf_sar.probs_to_labelme(test_mask_1, LABELS_FOLDER)

NameError: name 'test_mask' is not defined

In [15]:
class SentinelAcquisition:
    def __init__(self, data):
        self.column_name = "product_name"
        self.column_obg_product = "obj_product"
        self.column_product_path = "product_path"
        self.column_netcdf_path = "netcdf_path"
        self.column_obj_netcdf = "obj_netcdf"
        self.data = pd.DataFrame({self.column_name: data,
                                  self.column_obg_product: np.nan,
                                  self.column_product_path: np.nan,
                                  self.column_netcdf_path: np.nan,
                                  self.column_obj_netcdf: np.nan
                                 })

    def __str__(self):
        return self.data.to_string()

    def add_products(self, folder, nc_graph_path):
        for product in os.listdir(folder):
            if "SAFE" in product:
                product_name = product.split(".")[0]
                product_path = join(folder, product)
                search_result = self.nloc(product_name)
                if not search_result.empty:
                    product_obj = SentinelProduct(product_name, product_path, nc_graph_path)
                    self.data.loc[search_result.index, self.column_product_path] = product_path
                    self.data.loc[search_result.index, self.column_obg_product] = product_obj
                    
                    #display(self.data.loc[search_result.index])
                else:
                    print(product_name, "not found")
                    return 

    def download_products(self, folder):
        pending_download = self.data.loc[~self.data.index.isin(self.data.dropna(how="any").index)][self.column_name]
        print(pending_download)
        pending_download = list(pending_download)
        new_products = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=pending_download)
        new_products.download_products(folder=folder)

    def add_netcdfs(self, folder, image_variable='Sigma0_VV_db'):
        for nc_product in os.listdir(folder):
            nc_product_path = join(folder, nc_product)
            nc_name = nc_product.split(".")[0]
            search_result = self.nloc(nc_name)

    def to_netcdf4(self, folder):
        pass
        
        
#class NetcdfProduct:
#    def __init__(self, product, image_variable='Sigma0_VV_db'):
 
    
# class SentinelProduct:
#   def __init__(self, name, file_path, nc_graph_path,  rect_corners=None):               
        

    def __getitem__(self, index):
        return self.data.iloc[index]

    def nloc(self, name):
        output = self.data[self.data[self.column_name].str.contains(name)]
        if output.empty:
            print(name, "Not found!")
        else:  
            return output

In [19]:
SAR_TO_NC_GRAPH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/graphs/ZIP_to_NC.xml"
NETCDF_FOLDER ="/mnt/camobi_process/new_data/images_nc"
data_pd = pd.read_csv("/mnt/camobi_2/PHMG/New_sar_img.csv", header=0)[FILE_NAME_COLUMN]
data_pd = list(data_pd.dropna(how="all"))


FOLDER_PRODUCT = "/mnt/camobi_process/new_data/unzip_sar_img"

sar_data = SentinelAcquisition(data_pd)
sar_data.add_products("/mnt/camobi_process/new_data/unzip_sar_img", SAR_TO_NC_GRAPH)
sar_data.add_netcdfs(NETCDF_FOLDER)
#sar_data.download_products("/mnt/camobi_process/new_data/new_SAR_img")


C094 Not found!


In [20]:
sar_data.data.head(10)

,product_name,obj_product,product_path,netcdf_path,obj_netcdf
0,S1A_IW_SLC__1SDV_20181009T171427_20181009T1714...,NaN,NaN,NaN,NaN
1,S1B_IW_SLC__1SDV_20181008T172144_20181008T1722...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1B...,NaN,NaN
2,S1B_IW_SLC__1SDV_20210120T080527_20210120T0805...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1B...,NaN,NaN
3,S1A_IW_SLC__1SDV_20170810T024712_20170810T0247...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1A...,NaN,NaN
4,S1A_IW_SLC__1SDV_20170729T024712_20170729T0247...,<__main__.SentinelProduct object at 0x7efbe0b1...,/mnt/camobi_process/new_data/unzip_sar_img/S1A...,NaN,NaN
5,S1A_IW_SLC__1SDV_20210612T235346_20210612T2354...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1A...,NaN,NaN
6,S1B_IW_SLC__1SDV_20211003T014926_20211003T0149...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1B...,NaN,NaN
7,S1B_IW_SLC__1SDV_20211121T142946_20211121T1430...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1B...,NaN,NaN
8,S1A_IW_SLC__1SDV_20210805T000201_20210805T0002...,<__main__.SentinelProduct object at 0x7efbe0a7...,/mnt/camobi_process/new_data/unzip_sar_img/S1A...,NaN,NaN
9,S1A_IW_SLC__1SDV_20170709T000134_20170709T0002...,<__main__.SentinelProduct object at 0x7efbe0b2...,/mnt/camobi_process/new_data/unzip_sar_img/S1A...,NaN,NaN
